In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
def splitDate(date):
    date = str(date)
    Y = int(date[:4])
    M = int(date[4:6])
    D = int(date[6:])
    return (Y, M, D)

In [3]:
def getWeek(date):
    return datetime(*date).weekday()

In [4]:
read_csv = pd.read_csv("/Users/jooh8/Documents/GitHub/Quant/data/A000660/Index_A000660.txt", sep=" ", low_memory=False)

In [5]:
read_csv["DoW"] = read_csv["날짜"].apply(lambda x: getWeek(splitDate(x)))

In [6]:
weekList = []
preW = 0
week = 0

for W in reversed(read_csv["DoW"]):
    if W > preW:
        week += 1
    weekList.append(week)
    preW = W
    
weekList.reverse()

In [7]:
read_csv["WEEK"] = weekList

In [8]:
for period in range(10):
    
    read_csv["고가_{}".format(period)] = read_csv["고가"].shift(periods=-period)#.fillna(0)

In [9]:
read_csv = read_csv.dropna()

In [10]:
for period in range(10):
    
    read_csv["최고가_{}".format(period)] = read_csv[["고가_{}".format(i) for i in range(period + 1)]].apply(np.max, 1)

In [11]:
for period in range(10):
    
    read_csv["Target_{}".format(period)] = read_csv["시가"] * 1.05 < read_csv["고가_{}".format(period)]

In [12]:
columns = list(read_csv.columns)

In [13]:
momentums = columns[11:-32]
print(momentums)

['A Ratio_20', 'B Ratio_20', 'Aroon_20 Up', 'Down', 'Aroon Osillator_20', 'ATR_14', 'BB-RSI_종가_10', 'BB-RSI_20_2.00 상한', 'BB-RSI 하한', 'BPDL RSI_14', 'BPDL Stochatic_14', 'Chande Momentum Oscillator_10', "Chaikin's Volatility_14,14", 'CompuTrac Volatility_10', '+DI_14', '-DI_14', 'ADX_14', 'ADXR_14', '+DI(simple)_14', '-DI(simple)_14', 'ADX(simple)_14,14', 'ADXR(simple)_14,14,14', 'Energy+_14', 'Energy-_14', 'High Low Envelope_1', 'High Low Oscillator_3', 'Inertia_20,14,10', 'Klinger Oscillator', 'Linear Trend Oscillator_10,20', 'LRS_종가,14', 'Mass Index_20,9', 'Morris Mixed Momentum', 'New BPDL HiLo Index MA_14,7', 'Open Difference_15', 'QStick_14', 'Random Walk Index_15,3', 'RCI_5', 'RCI_9', 'RCI_13', 'RCI_18', 'Relative Volatility Index_단순,14,10', 'Reverse 단기_12', ' 장기_24', 'RSI_종가,14', 'RSI(simple)_종가,14', 'Sigma_종가,20', 'SMI_5,3,3', 'Standard Deviation_14,2.00', 'Standard Error_종가,14', 'Fast %K_5', 'Fast %D_3', 'Slow %K_5,3', 'Slow %D_3', 'Slow(Simple) %K_5,3', 'Slow(Simple) %D_3', 

In [14]:
targets = columns[-10:]
print(targets)

['Target_0', 'Target_1', 'Target_2', 'Target_3', 'Target_4', 'Target_5', 'Target_6', 'Target_7', 'Target_8', 'Target_9']


In [15]:
DATASET = []

for idx in tqdm(range(3, 100)):
    
    X_train = read_csv.loc[(read_csv["WEEK"] > idx+3) & (read_csv["WEEK"] < idx+503), momentums]
    Y_train = read_csv.loc[(read_csv["WEEK"] > idx+3) & (read_csv["WEEK"] < idx+503), targets]
    
    TESTSET = []

    X_test = read_csv.loc[(read_csv["WEEK"] == idx+3), momentums]
    Y_test = read_csv.loc[(read_csv["WEEK"] == idx+3), targets]
    
    TESTSET.append([X_test, Y_test])
    
    X_test = read_csv.loc[(read_csv["WEEK"] == idx+2), momentums]
    Y_test = read_csv.loc[(read_csv["WEEK"] == idx+2), targets]
    
    TESTSET.append([X_test, Y_test])
    
    X_test = read_csv.loc[(read_csv["WEEK"] == idx+1), momentums]
    Y_test = read_csv.loc[(read_csv["WEEK"] == idx+1), targets]
    
    TESTSET.append([X_test, Y_test])
    
    X_test = read_csv.loc[(read_csv["WEEK"] == idx), momentums]
    Y_test = read_csv.loc[(read_csv["WEEK"] == idx), targets]
    
    TESTSET.append([X_test, Y_test])
    
    DATASET.append([[X_train, Y_train], TESTSET])

100%|███████████████████████████████████████████████████████| 97/97 [00:02<00:00, 44.16it/s]


In [16]:
DATASET[0][1][3][0]

,A Ratio_20,B Ratio_20,Aroon_20 Up,Down,Aroon Osillator_20,ATR_14,BB-RSI_종가_10,BB-RSI_20_2.00 상한,BB-RSI 하한,BPDL RSI_14,...,"Slow %K_5,3",Slow %D_3,"Slow(Simple) %K_5,3",Slow(Simple) %D_3,"StochOsc_5,3","TRIX기울기_종가,14",True Range,"TSF Oscillator_종가,14,28",VHF_14,CCI_14
5604,108.0568720379147,139.89071038251367,70.0,0.0,70.0,2384.0206562523485,55.58355681401927,74.66372192483402,51.36534979012427,56.707317073170735,...,46.666666666666664,41.988231436903334,47.51175705726277,45.491084886481524,11.865211200759305,-0.01771295080341545,1900.0,-1297.8021978024044,0.22560975609756098,-15.876656472947905
5605,116.50485436893203,126.3959390862944,65.0,0.0,65.0,2399.447752234323,51.27040144260923,74.94323828132495,49.65296877599513,51.204819277108435,...,40.476190476190474,40.395136778115244,42.505878528631385,43.99848170755645,-2.9069767441861174,-0.02089261704175921,2600.0,-1689.6825396823406,0.22289156626506024,-39.678510998266475
5606,107.17703349282297,119.79695431472082,60.0,25.0,35.0,2385.201484217586,57.49980045832649,73.23524754788421,49.776308067450074,56.353591160220994,...,58.19672131147541,48.446526151443926,60.99652900790544,52.497505357730944,21.111608030212615,-0.01747649414303315,2200.0,-1878.3272283273836,0.20441988950276244,40.57971014497359
5607,113.0890052356021,126.11111111111111,55.00000000000001,20.0,35.00000000000001,2286.258521059187,56.093760395781935,72.7423331583389,49.25435416790536,56.353591160220994,...,63.793103448275865,54.15533841198033,68.33610234179056,60.416803849760754,11.454723954723889,-0.016168800832854546,1000.0,-1947.9242979245319,0.18232044198895028,59.74625144180584


In [17]:
DATASET[0][1][3][1]

,Target_0,Target_1,Target_2,Target_3,Target_4,Target_5,Target_6,Target_7,Target_8,Target_9
5604,False,False,False,False,False,False,False,False,False,False
5605,False,False,False,False,False,False,False,False,False,False
5606,False,False,False,False,False,False,False,False,False,False
5607,False,False,False,False,False,False,False,False,False,False


In [18]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
import numpy as np

In [19]:
CMResult = {}

In [20]:
for date, data in enumerate(tqdm(DATASET)):
    
    if date % 10 != 0:
        continue
    
    X_train = data[0][0].astype(float)
    Y_train = data[0][1].astype(float)
    
    for week, test in enumerate(data[1]):
        
        X_test = test[0].astype(float).astype(float)
        Y_test = test[1].astype(float)

        scaler = preprocessing.StandardScaler().fit(X_train)
        forest = RandomForestClassifier(n_estimators=500, random_state=1, max_depth=5)
        multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
        multi_target_forest.fit(scaler.transform(X_train), Y_train)
        predict = multi_target_forest.predict(scaler.transform(X_train))

        # corr = predict == Y_train
        # corr.apply(np.mean, 0)
        # np.mean(corr.apply(np.mean, 0))

        predict = multi_target_forest.predict(scaler.transform(X_test))

        # corr = predict == Y_test
        # corr.apply(np.mean, 0)
        # np.mean(corr.apply(np.mean, 0))
        
        for period in range(10):

            cm = confusion_matrix(np.array(Y_test["Target_{}".format(period)]), predict[:, period])
            if len(cm) == 1:
                
                tmp = int(list(set(predict[:, period]))[0])
                values = int(cm)
                
                if tmp == 0:
                    cm = np.zeros((2, 2))
                    cm[0, 0] = values
                    
                else:
                    cm = np.ones((2, 2))
                    cm[1, 1] = values

    #         Sensitivity = cm[1, 1] / np.sum(cm[:, 1])
    #         Accuracy = np.sum(cm.diagonal()) / np.sum(cm)
    
            key = "week{}_period{}".format(week+1, period)    
            
            if key not in CMResult:
                CMResult[key] = []

            CMResult[key].append(cm)

100%|███████████████████████████████████████████████████████| 97/97 [05:15<00:00,  3.25s/it]


In [21]:
CMResult.keys()

dict_keys(['week1_period0', 'week1_period1', 'week1_period2', 'week1_period3', 'week1_period4', 'week1_period5', 'week1_period6', 'week1_period7', 'week1_period8', 'week1_period9', 'week2_period0', 'week2_period1', 'week2_period2', 'week2_period3', 'week2_period4', 'week2_period5', 'week2_period6', 'week2_period7', 'week2_period8', 'week2_period9', 'week3_period0', 'week3_period1', 'week3_period2', 'week3_period3', 'week3_period4', 'week3_period5', 'week3_period6', 'week3_period7', 'week3_period8', 'week3_period9', 'week4_period0', 'week4_period1', 'week4_period2', 'week4_period3', 'week4_period4', 'week4_period5', 'week4_period6', 'week4_period7', 'week4_period8', 'week4_period9'])

In [22]:
for i in CMResult["week1_period9"]:
    print(i)

[[2 0]
 [3 0]]
[[4 0]
 [1 0]]
[[3 0]
 [2 0]]
[[5. 0.]
 [0. 0.]]
[[1 0]
 [4 0]]
[[4 0]
 [1 0]]
[[2 0]
 [1 0]]
[[5. 0.]
 [0. 0.]]
[[0 0]
 [4 1]]
[[2 0]
 [3 0]]
